In [1]:
import netCDF4 as nc
import numpy as np
import pandas as pd

In [117]:
def set_source_ids(n_modes, n_aero_sources):
    source_ids = []
    modes_per_source = int(n_modes/n_aero_sources)

    for i in np.arange(1, 1+n_aero_sources):
        for j in range(modes_per_source):
            source_ids.append(i)
    return np.array(source_ids)

def set_char_radii(n_modes, n_aero_sources):
    char_radii = []
    modes_per_source = int(n_modes/n_aero_sources)

    if modes_per_source == 2:
        mode_char_radii = [1.8090e-08, 3.9905e-08] # using defaults from Jeff's aero_emit_dists_001_001_001.nc file
    else:
        raise ValueError(f'Undefined mode radii for number of modes per source: {modes_per_source}')

    for i in np.arange(1, 1+n_aero_sources):
        for radius in mode_char_radii:
            char_radii.append(radius)
    return np.array(char_radii)


def set_log10_std_dev_radius(n_modes, n_aero_sources):
    log10_std_dev_radii = []
    modes_per_source = int(n_modes/n_aero_sources)

    if modes_per_source == 2:
        mode_stddev_radii = [0.20411998, 0.25527251] # using defaults from Jeff's aero_emit_dists_001_001_001.nc file
    else:
        raise ValueError(f'Undefined mode std dev for number of modes per source: {modes_per_source}')

    for i in np.arange(1, 1+n_aero_sources):
        for radius in mode_stddev_radii:
            log10_std_dev_radii.append(radius)
    return np.array(log10_std_dev_radii)

def set_num_conc(n_times, n_modes):
    # setting number concentration of each mode to zero at each time
    num_conc = np.zeros((n_times, n_modes))
    #for i in range(n_times):
    #    num_conc[i] = np.zeros(n_modes) 
    return num_conc

def set_vol_frac(n_times, n_modes, n_aero_specs):

    vol_frac = np.zeros((n_times, n_modes, n_aero_specs))

    for itime in range(n_times):
        for imode in range(n_modes):
            for ispecies in range(n_aero_specs):
                # For now, just set the volume fraction to be equal across all species
                # Makes it easy to ensure that volume fractions add up to one
                spec_vol_frac = 1 / n_aero_specs
                vol_frac[itime, imode, ispecies] = spec_vol_frac

    return vol_frac

def set_gas_emission(n_times, n_gas_specs):

    # Using emission rates from center of CARES domain, aero_emit_dists_085_080_001.nc, variable name 'gas_emission', t index = 0
    emiss_data = pd.read_csv('/data/keeling/a/sf20/b/wrf_partmc/WRFV3/test/em_les/cares_gas_emiss_rates.csv')
    emission_rates = np.zeros((n_times, n_gas_specs))

    for itime in range(n_times):
        for ispec in range(n_gas_specs):
            # NOTE: set all gas phase emission rates to zero for now
            spec_emiss_rate = emiss_data.loc[ispec, 'Emission rate (mol m^{-2} s^{-1})']
            emission_rates[itime, ispec] = spec_emiss_rate
    return emission_rates


In [118]:
file_path = '/data/keeling/a/sf20/b/wrf_partmc/WRFV3/test/em_les/test.nc'
ncfile = nc.Dataset(file_path, 'w')

# Dimension values
n_times = 25
n_modes = 2
n_aero_specs = 20
n_aero_sources = 1
n_gas_specs = 77

one_hour = 3600

# Create dimensions
times_dim = ncfile.createDimension('n_times', n_times)
modes_dim = ncfile.createDimension('n_modes', n_modes)
aero_specs_dim = ncfile.createDimension('n_aero_specs', n_aero_specs)
aero_sources_dim = ncfile.createDimension('n_aero_sources', n_aero_sources)
gas_specs_dim = ncfile.createDimension('n_gas_specs', n_gas_specs)

# Create variables and set values
aero_emission_rate_scale = ncfile.createVariable('aero_emission_rate_scale', 'f8', ('n_times'))
aero_emission_rate_scale.unit = '(1)'
aero_emission_rate_scale.long_name = "Aerosol emission rate"
aero_emission_rate_scale.description = "Aerosol emission rate scales at set-points"
aero_emission_rate_scale[:] = 1 # set all rates to 1

aero_emission_time = ncfile.createVariable('aero_emission_time', 'f8', ('n_times'))
aero_emission_time.unit = 's'
aero_emission_time.long_name = "Aerosol emission time"
aero_emission_time.description = "Aerosol emission set-points times (s)."
aero_emission_time[:] = np.arange(0, n_times*one_hour, one_hour) # set times to each hour during a 24hr period

# Using Jeff's presets for characteristic radii
char_radius = ncfile.createVariable('char_radius', 'f8', ('n_modes'))
char_radius.unit = 'm'
char_radius.long_name = "characteristic_radius"
char_radius.standard_name = "characteristic_radius"
char_radius.description = "Characteristic radius, with meaning dependent on mode type"
char_radius[:] = set_char_radii(n_modes, n_aero_sources)

# Using Jeff's presets for log10 std dev of radius
log10_std_dev_radius = ncfile.createVariable('log10_std_dev_radius', 'f8', ('n_modes'))
log10_std_dev_radius.unit = 'm'
log10_std_dev_radius.long_name = "log10_std_dev_radius"
log10_std_dev_radius.standard_name = "log10_std_dev_radius"
log10_std_dev_radius.description = "Log base 10 of geometric standard deviation of radius, (m)."
log10_std_dev_radius[:] = set_log10_std_dev_radius(n_modes, n_aero_sources)

source_id = ncfile.createVariable('source_id', 'i', ('n_modes'))
source_id.unit = '(1)'
source_id.long_name = "Source number."
source_id.standard_name = "Source number"
source_id.description = "Source number for each emission mode."
source_id[:] = set_source_ids(n_modes, n_aero_sources)

# NOTE: currently setting number concentration to zero!
# TODO: Modify for non-zero aerosol emissions!
num_conc = ncfile.createVariable('num_conc', 'f8', ('n_times', 'n_modes'))
num_conc.unit = '# m^{-3}'
num_conc.long_name = "total number concentration"
num_conc.standard_name = "total number concentration"
num_conc.description = "Total number concentration of mode (#/m^3)."
num_conc[:] = set_num_conc(n_times, n_modes)

vol_frac = ncfile.createVariable('vol_frac', 'f8', ('n_times', 'n_modes', 'n_aero_specs'))
vol_frac.unit = '(1)'
vol_frac.long_name = "species fractions"
vol_frac.standard_name = "species_fractions"
vol_frac.description = "Species fractions by volume [length \\c aero_data%%n_spec]."
vol_frac[:] = set_vol_frac(n_times, n_modes, n_aero_specs)

# NOTE: Not entire sure what values I should assign here
source_weight_class = ncfile.createVariable('source_weight_class', 'i', ('n_modes'))
source_weight_class.unit = '(1)'
source_weight_class.long_name = "Aerosol weight class"
source_weight_class.description = "Weight class ID for each aerosol mode."
source_weight_class[:] = 1 # just set all wieght classes to 1 for now

aero_source = ncfile.createVariable('aero_source', 'i', ('n_aero_sources'))
aero_source.names = 'ideal_source'
aero_source.description = "dummy dimension variable (no useful value) - read source names as comma-separated values from the \'names\' attribute"
aero_source[:] = np.arange(1, n_aero_sources+1, 1)

# NOTE: Jeff has these values set to 5e-2 for most gas species for some reason?
gas_emission_rate_scale = ncfile.createVariable('gas_emission_rate_scale', 'f8', ('n_times'))
gas_emission_rate_scale.unit = '(1)'
gas_emission_rate_scale.long_name = "Gas emission rate scale factor"
gas_emission_rate_scale.description = "Gas emission rate scales at set-points"
gas_emission_rate_scale[:] = 1 # set all rates to 1

gas_emission_time = ncfile.createVariable('gas_emission_time', 'f8', ('n_times'))
gas_emission_time.unit = 's'
gas_emission_time.long_name = "Gas emission time"
gas_emission_time.description = "Gas emission set-points times (s)."
gas_emission_time[:] = np.arange(0, n_times*one_hour, one_hour) # set times to each hour during a 24hr period

# NOTE: set using values in center of CARES domain, will need case for zero emission (simple, just set array all zero)
gas_emission = ncfile.createVariable('gas_emission', 'f8', ('n_times', 'n_gas_specs'))
gas_emission.unit = 'mol m^{-2} s^{-1}'
gas_emission.long_name = "gas emissions"
gas_emission.standard_name = "gas emissions"
gas_emission.description = "gas phase emission rates."
gas_emission[:] = set_gas_emission(n_times, n_gas_specs)

ncfile.close()


In [2]:
#path = '/data/keeling/a/sf20/b/wrf_partmc/WRFV3/test/em_les/CARES_aero_emit_dists_001_001_001.nc'
path = '/data/keeling/a/jcurtis2/d/wrf_partmc_keeling_dev/wrf_partmc/emissions/cares_emissions_json/aero_emit_dists_085_080_001.nc'
data = nc.Dataset(path)

In [7]:
data

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): n_times(49), n_modes(100), n_aero_specs(20), n_aero_sources(50), n_gas_specs(77)
    variables(dimensions): float64 aero_emission_rate_scale(n_times), float64 aero_emission_time(n_times), float64 char_radius(n_modes), float64 log10_std_dev_radius(n_modes), int32 source_id(n_modes), float64 num_conc(n_times, n_modes), float64 vol_frac(n_times, n_modes, n_aero_specs), int32 source_weight_class(n_modes), int32 aero_source(n_aero_sources), float64 gas_emission_rate_scale(n_times), float64 gas_emission_time(n_times), float64 gas_emission(n_times, n_gas_specs)
    groups: 

In [13]:
data['num_conc']

<class 'netCDF4._netCDF4.Variable'>
float64 num_conc(n_times, n_modes)
    unit: # m^{-3}
    long_name: total number concentration
    standard_name: total number concentration
    description: Total number concentration of mode (#/m^3).
unlimited dimensions: 
current shape = (49, 100)
filling on, default _FillValue of 9.969209968386869e+36 used

In [12]:
data['num_conc'][0, :]

masked_array(data=[0.00000000e+00, 0.00000000e+00, 7.39081411e+00,
                   7.43530644e+02, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 1.42814251e+03, 1.43673982e+05,
                   4.90894592e+03, 2.52785212e+06, 2.92782505e+02,
                   1.50767788e+05, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
                   0.00000000e+00, 1.70519619e+04, 2.14011574e+05,
                   0.00000000e+00, 0.00000000e+00, 1.61503502e+02,
                   1.57900240e+02, 1.31540469e+03, 3.81533715e+03,
                   1.56797462e+03, 4.39125440e+03, 0.00000000e

In [5]:
data['aero_source']

<class 'netCDF4._netCDF4.Variable'>
int32 aero_source(n_aero_sources)
    names: agr_fires,aircraft,biogenic,cooking,egu_biomass,egu_coalBrown,egu_coalHard_nop,egu_coalHard_pul,egu_natgas,egu_oil,egu_other,nonroad_diesel,nonroad_gas2strk,nonroad_gas4strk,nonroad_other,np_agr,np_coal,np_fdust,np_gasops,np_indus_oil,np_indus_wood,np_natgas,np_oilgas,np_other,np_rail,onroad_hddiesel,onroad_hdgas,onroad_lddiesel,onroad_ldgas,onroad_other,pt_biomass,pt_coal,pt_fdust,pt_gasops,pt_indus_wood,pt_inuds_oil,pt_natgas,pt_oilgas,pt_other,pt_rail,res_diesel,res_fireplace,res_furnace,res_openfire,res_other,res_woodstove,rx_fires,ship_diesel,ship_int,wild_fires
    description: dummy dimension variable (no useful value) - read source names as comma-separated values from the 'names' attribute
unlimited dimensions: 
current shape = (50,)
filling on, default _FillValue of -2147483647 used

In [122]:
test_data = nc.Dataset('test.nc')

In [123]:
test_data['gas_emission'][0]

masked_array(data=[0.00e+00, 0.00e+00, 0.00e+00, 4.86e-08, 2.21e-09,
                   1.88e-10, 0.00e+00, 0.00e+00, 1.63e-11, 0.00e+00,
                   0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
                   0.00e+00, 2.82e-08, 3.76e-12, 9.19e-10, 1.28e-10,
                   0.00e+00, 0.00e+00, 2.23e-10, 8.93e-13, 2.62e-10,
                   0.00e+00, 0.00e+00, 1.13e-10, 0.00e+00, 0.00e+00,
                   0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
                   0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
                   5.05e-09, 3.04e-12, 0.00e+00, 3.93e-10, 3.54e-10,
                   2.46e-10, 1.40e-10, 1.21e-10, 0.00e+00, 0.00e+00,
                   0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
                   0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 2.22e-13,
                   0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
                   0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00, 0.00e+00,
                   0.00e+00, 0.00e

In [ ]:
fill('')